In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import optimize

In [2]:
dataframe = pd.read_csv("./FATS_GAIA.dat")
dataframe = dataframe.drop(["Class", "ID"], axis=1)
dataframe.head()

,Amplitude,AndersonDarling,Autocor_length,Con,Eta_e,FluxPercentileRatioMid20,FluxPercentileRatioMid35,FluxPercentileRatioMid50,FluxPercentileRatioMid65,FluxPercentileRatioMid80,...,PeriodLS,Period_fit,Psi_CS,Psi_eta,Q31,Rcs,Skew,SlottedA_length,SmallKurtosis,Std
0,1.416432,0.949482,3.0,0.000000,5.442293,0.243054,0.462373,0.653511,0.761790,0.924793,...,268.331543,0.025528,0.304804,0.310355,1.784879,0.304804,0.011895,19.899170,-1.266215,0.931676
1,0.443386,1.000000,1.0,0.000000,12042.752633,0.162446,0.231268,0.517775,0.720122,0.836798,...,0.508910,0.012390,0.309458,0.519645,0.397205,0.258203,-0.962335,0.147949,-0.183573,0.284637
2,0.170099,0.875986,4.0,0.030303,137.020266,0.033964,0.208724,0.378623,0.501416,0.869956,...,8.742769,0.040022,0.342646,0.700200,0.119849,0.368936,0.472161,0.147949,0.064528,0.089020
3,1.350858,0.999869,4.0,0.000000,23.583559,0.259476,0.593270,0.613808,0.817245,0.943033,...,379.949707,0.000843,0.349687,0.285682,1.867060,0.349687,-0.047072,23.597900,-1.454570,0.926506
4,0.248472,0.999947,3.0,0.000000,125.486491,0.202240,0.448444,0.565120,0.606539,0.840512,...,318.427795,0.022169,0.247451,0.578087,0.273482,0.247451,0.594605,0.147949,-0.689124,0.153349


## Preprocesamiento: dividir base de datos en *train* y *test*

In [3]:
train = dataframe.sample(frac=0.8, random_state=7).reset_index()
train = train.drop("index", 1)
test = dataframe.drop(train.index).reset_index()
test = test.drop("index", 1)

In [4]:
X_train = train[[column for column in train.columns if column != "PeriodLS"]]
y_train = train["PeriodLS"]
X_test = test[[column for column in train.columns if column != "PeriodLS"]]
y_test = test["PeriodLS"]

In [5]:
display(X_train.head())
display(y_train.head())
display(X_test.head())
display(y_test.head())

,Amplitude,AndersonDarling,Autocor_length,Con,Eta_e,FluxPercentileRatioMid20,FluxPercentileRatioMid35,FluxPercentileRatioMid50,FluxPercentileRatioMid65,FluxPercentileRatioMid80,...,PercentDifferenceFluxPercentile,Period_fit,Psi_CS,Psi_eta,Q31,Rcs,Skew,SlottedA_length,SmallKurtosis,Std
0,0.165094,0.711871,2.0,0.0,149.154338,0.286535,0.315403,0.354876,0.634872,0.817463,...,0.022645,0.585891,0.267306,1.110589,0.106045,0.345638,0.111140,28.406250,-0.350345,0.092656
1,0.233673,0.999928,1.0,0.0,84885.514569,0.175111,0.320062,0.454982,0.524935,0.817864,...,0.024830,0.999908,0.347286,1.039225,0.206278,0.233412,-0.874187,0.073975,-0.152506,0.152241
2,0.490221,0.999994,1.0,0.0,17974.650153,0.329906,0.394319,0.508053,0.632579,0.845557,...,0.059739,0.005520,0.281397,0.759796,0.447899,0.197481,-0.641018,0.073975,-0.613068,0.293194
3,0.538722,1.000000,1.0,0.0,21720.314290,0.209576,0.400352,0.481303,0.588752,0.959153,...,0.065208,0.000047,0.288827,0.319746,0.522474,0.122131,-0.722468,0.073975,-0.730431,0.353988
4,0.256679,0.951044,1.0,0.0,14413.913756,0.235947,0.332411,0.451095,0.547342,0.739669,...,0.028744,0.078749,0.325063,0.708258,0.210350,0.308932,-0.609829,0.073975,-0.205571,0.155104


0    1.224964
1    3.954775
2    0.529830
3    0.482072
4    0.867182
Name: PeriodLS, dtype: float64

,Amplitude,AndersonDarling,Autocor_length,Con,Eta_e,FluxPercentileRatioMid20,FluxPercentileRatioMid35,FluxPercentileRatioMid50,FluxPercentileRatioMid65,FluxPercentileRatioMid80,...,PercentDifferenceFluxPercentile,Period_fit,Psi_CS,Psi_eta,Q31,Rcs,Skew,SlottedA_length,SmallKurtosis,Std
0,0.327284,0.991928,1.0,0.0,58412.807332,0.152161,0.227720,0.426300,0.598559,0.701074,...,0.033710,0.395342,0.338043,0.974978,0.303738,0.185923,-0.768070,0.073975,0.206281,0.186596
1,0.257141,0.698589,1.0,0.0,17613.263062,0.151067,0.235604,0.477704,0.705610,0.939053,...,0.025574,0.053945,0.316196,0.496841,0.250239,0.173091,-0.080403,0.147949,-0.799777,0.155838
2,0.221411,0.999640,1.0,0.0,44426.219390,0.416797,0.524806,0.699892,0.818684,0.976095,...,0.022104,0.213965,0.326893,0.568963,0.293938,0.185099,-0.039193,0.073975,-1.558781,0.156909
3,0.202297,0.999475,1.0,0.0,28565.796345,0.286940,0.576348,0.650045,0.851497,0.918296,...,0.020635,0.429962,0.245748,1.199581,0.283047,0.205665,-0.236193,0.073975,-1.387901,0.141242
4,0.230783,0.999916,1.0,0.0,46618.997454,0.175520,0.341385,0.539694,0.895385,0.941805,...,0.026930,0.948142,0.278585,1.516252,0.281733,0.236349,-0.477540,0.073975,-1.041804,0.160290


0    0.546369
1    0.831368
2    0.829974
3    0.951642
4    0.490445
Name: PeriodLS, dtype: float64

## Construcción del árbol

In [6]:
class Node:
    id_count = 0

    def __init__(self, X, y, parent=None):
        self.X = X
        self.y = y
        self.parent = parent
        self.id = Node.id_count
        self.left = None
        self.right = None
        Node.id_count += 1
        self.children = dict()


    def find_split(self):
        total_node_variance = self.y.var()
        features_var_red = {}
        for feature in self.X.columns:
            print(feature)
            feature_values = self.X[feature].unique()
            values_var_red = []
            for value in feature_values:
                left_side = self.X[self.X[feature] <= value]
                right_side = self.X[self.X[feature] > value]
                left_side_var = self.y[self.y.index.isin(left_side.index)]
                right_side_var = self.y[self.y.index.isin(right_side.index)]
                final_var = total_node_variance - \
                    (left_side_var + right_side_var)
                values_var_red.append(final_var)
            max_var_red = max(values_var_red)
            features_var_red[feature] = max_var_red
        return max(features_var_red, key=features_var_red.get)

    @staticmethod
    def var_red_function(value, feature, X_values, y_values):
        left_side = X_values[X_values[feature] <= value]
        right_side = X_values[X_values[feature] > value]
        left_side_y = y_values[y_values.index.isin(left_side.index)]
        right_side_y = y_values[y_values.index.isin(right_side.index)]
        return left_side_y.var() + right_side_y.var()
    
    @staticmethod
    def calculate_rss(y_values):
        return np.sum((y_values - y_values.mean()) ** 2)
    
    @staticmethod
    def rss_function(value, feature, X_values, y_values):
        left_side = X_values[X_values[feature] <= value]
        right_side = X_values[X_values[feature] > value]
        left_side_y = y_values[y_values.index.isin(left_side.index)]
        right_side_y = y_values[y_values.index.isin(right_side.index)]
        return Node.calculate_rss(left_side_y) + Node.calculate_rss(right_side_y)
        
    def split_with_scipy(self):
        features_var_red = dict()
        for feature in self.X.columns:
            feature_values = self.X[feature].unique()
            if min(feature_values) == max(feature_values):
                continue
            optimal_split, split_var_red, ierr, numf = scipy.optimize.fminbound(
                Node.rss_function,
                min(feature_values),
                max(feature_values),
                args=(feature, self.X, self.y),
                full_output=1)
            features_var_red[feature] = (split_var_red, optimal_split)
        best_split = sorted(features_var_red.items(), key=lambda x: x[1])[0]
        self.split_label = best_split[0]
        self.split_value = best_split[1][-1]
        return self.split_label, self.split_value

In [7]:
def fit(data, target, parent, depth, max_depth=2, min_values=20):
    # print("Current depth: {}".format(depth))
    root = Node(data, target, parent)
    # print("There are {} rows in current node".format(len(root.X)))
    if len(root.X) < min_values:
        # print("Se alcanzó mínimo de valores. El nodo {} es un nodo hoja".format(root.id))
        return root
    if depth >= max_depth:
        # print("Se alcanzó altura máxima. El nodo {} es un nodo hoja".format(root.id))
        return root
    
    root.split_with_scipy()
    split_label = root.split_label
    split_value = root.split_value
    # print(split_label, split_value)
    left_data = root.X[root.X[split_label] <= split_value]
    right_data = root.X[root.X[split_label] > split_value]
    left_target = root.y[root.y.index.isin(left_data.index)]
    right_target = root.y[root.y.index.isin(right_data.index)]
    
    # print("Creando hijo izquierdo del nodo {}".format(root.id))
    root.left = fit(left_data, left_target, root, depth + 1, max_depth, min_values)
    # print("Creando hijo derecho del nodo {}".format(root.id))
    root.right = fit(right_data, right_target, root, depth + 1, max_depth, min_values)
    return root

In [8]:
node = fit(X_train, y_train, None, 0)

In [12]:
def predict_datum(root, datum):
    if root.left is None:
        return root.y.mean()
    if datum.loc[root.split_label] <= root.split_value:
        return predict_datum(root.left, datum)
    return predict_datum(root.right, datum)

In [17]:
def predict(root, test_data):
    return test_data.apply(lambda x: predict_datum(root, x), axis=1)

In [19]:
y_pred = predict(node, X_test)

In [21]:
print(y_pred)

0          2.782821
1          2.782821
2          2.782821
3          2.782821
4          2.782821
5          2.782821
6          2.782821
7          2.782821
8          2.782821
9          2.782821
10         2.782821
11         2.782821
12         2.782821
13         2.782821
14         2.782821
15         2.782821
16         2.782821
17         2.782821
18         2.782821
19        92.085742
20       136.118192
21         2.782821
22         2.782821
23         2.782821
24         2.782821
25        92.085742
26         2.782821
27         2.782821
28        92.085742
29        92.085742
            ...    
40104      2.782821
40105      2.782821
40106      2.782821
40107      2.782821
40108      2.782821
40109      2.782821
40110      2.782821
40111      2.782821
40112      2.782821
40113      2.782821
40114      2.782821
40115      2.782821
40116      2.782821
40117      2.782821
40118      2.782821
40119      2.782821
40120    136.118192
40121      2.782821
40122      2.782821


In [22]:
def calculate_rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [24]:
rmse = calculate_rmse(y_pred, y_test)
print(rmse)

140.74091548170497
